In [1]:
import numpy as np
import networkx as nx
from operator import itemgetter
import import_ipynb
import QAOA_for_Recursive as qaoa

importing Jupyter notebook from QAOA_for_Recursive.ipynb


In [2]:
def recursive_QAOA(G,k, iter=0):
    if len(G)==0:
        return 0
    result=qaoa.qaoa_basic(len(G),G)
    solution=result.x
        
    for i,j in zip(G.nodes, range(0,len(G))):
        G.nodes[i]["Spin{}".format(iter)]=solution[j]
        
    subnodes1 = (node
                    for node, data
                    in G.nodes(data=True)
                    if data.get("Spin{}".format(iter)) == 1)
        
    subnodes2 = (node
                    for node, data
                    in G.nodes(data=True)
                    if data.get("Spin{}".format(iter)) == 0)
        
    G1 = G.subgraph(subnodes1)
    G2 = G.subgraph(subnodes2)
    
    
    iter+=1
    if iter>=k:
        return 0
    
    recursive_QAOA(G1,k,iter)
    recursive_QAOA(G2,k,iter)

In [3]:
G=nx.barbell_graph(4,0)
G.add_nodes_from([9,10,11])
E=[(1,9,0.001),(9,10,5),(10,11,5),(9,11,5)]
G.add_weighted_edges_from(E)

#nx.draw_networkx(G)
recursive_QAOA(G,2)

optimal function value: 13.928073979500732
optimal value: [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]
status: SUCCESS
optimal function value: 0.0
optimal value: [0. 0. 0.]
status: SUCCESS
optimal function value: 5.5
optimal value: [1. 1. 1. 1. 0. 0. 0. 0.]
status: SUCCESS


In [28]:
def compute_mod(G,k):
    for i in G.nodes:
        sum=0
        for j in range(0,k):
            sum=sum+pow(2,j)*G.nodes[i]["Spin{}".format(k-1-j)]
        G.nodes[i]["colour"]=sum
    
    groups = set(nx.get_node_attributes(G,'colour').values())
    communities=list()
    
    for l in groups:
        group=set()
        for i in G.nodes:
            if G.nodes[i]["colour"]==l:
                group.add(i)
        communities.append(group)
    
    result=nx.algorithms.community.quality.modularity(G, communities)
    return result, communities

In [29]:
compute_mod(G,2)

(0.5694816428042364, [{4, 5, 6, 7}, {0, 1, 2, 3}, {9, 10, 11}])